In [2]:
desired_system_capacity = '800'
option_table_path = 'example-diesel-generator-by-capacity.csv'
capacity_column = 'capacity_in_kw'

In [3]:
from pandas import read_csv
option_table = read_csv(option_table_path)
option_table

,Capacity in kW,Raw Cost,Installation Cost as Percent of Raw Cost,Maintenance Cost per Year as Percent of Raw Cost,Lifetime in Years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [5]:
from invisibleroads_macros.table import normalize_column_name
[normalize_column_name(x) for x in option_table.columns]

['capacity in kw',
 'raw cost',
 'installation cost as percent of raw cost',
 'maintenance cost per year as percent of raw cost',
 'lifetime in years']

In [8]:
normalize_column_name('kWh')

'kwh'

In [ ]:
def prepare_system_cost(desired_system_capacity, option_table, capacity_column):
    

In [ ]:
"""
    Name initial variables
    Draft table
    Specify final variables

    Fix oversizing system capacity
        Draft system capacity algorithm
            Have user specify generator capacity
            Have user specify generator raw cost
            Have user specify generator installation cost as fraction of raw cost
            Have user specify maintenance cost per year as fraction of raw cost
            Get desired system capacity
            _ Add system capacity safety factor
            Compute raw cost per kw
            Pick nearest generator capacity and use those costs per kw
            Round actual capacity to integer
"""

In [ ]:
def prepare_system_cost(
        desired_system_capacity, option_table, capacity_column):
    t = option_table
    # Select option
    eligible_t = t[t[capacity_column] < desired_system_capacity]
    if len(eligible_t):
        t = eligible_t
        # Choose the largest capacity from eligible types
        selected_t = t[t[capacity_column] == t[capacity_column].max()]
    else:
        # Choose the smallest capacity from all types
        selected_t = t[t[capacity_column] == t[capacity_column].min()]
    selected = selected_t.ix[selected_t.index[0]]
    # Get capacity and count
    selected_capacity = selected[capacity_column]
    selected_count = int(ceil(divide_safely(
        desired_system_capacity, selected_capacity,
        ExpectedPositive('selected_capacity'))))
    actual_system_capacity = selected_capacity * selected_count
    # Get costs
    installation_lm_cost = selected_count * selected['installation_lm_cost']
    maintenance_lm_cost_per_year = selected_count * selected[
        'maintenance_lm_cost_per_year']
    replacement_lm_cost_per_year = divide_safely(
        installation_lm_cost, selected['lifetime_in_years'],
        ExpectedPositive('lifetime_in_years'))
    return {
        'desired_system_' + capacity_column: desired_system_capacity,
        'selected_' + capacity_column: selected_capacity,
        'selected_count': selected_count,
        'actual_system_' + capacity_column: actual_system_capacity,
        'installation_lm_cost': installation_lm_cost,
        'maintenance_lm_cost_per_year': maintenance_lm_cost_per_year,
        'replacement_lm_cost_per_year': replacement_lm_cost_per_year,
    }